# Predict

In [1]:
# Parameters cells
WITNESS_NAME = "CHSH"
SIMULATION_PATH = "./simulated_data"
MLFLOW_URL = "http://localhost:5000"
AIRFLOW_DAG_RUN_ID = "304d8395-d05b-41af-bfff-4a8af2f3bf47-CHSH"
MLFLOW_RUN_ID = None #"1c81e50024c54dd69d8e48bc406c4dad"
AWS_ACCESS_KEY_ID="minio123"
AWS_SECRET_ACCESS_KEY="minio123"
MLFLOW_S3_ENDPOINT_URL="http://localhost:9990"

In [2]:
from os import environ

environ["AWS_ACCESS_KEY_ID"] = AWS_ACCESS_KEY_ID
environ["AWS_SECRET_ACCESS_KEY"] = AWS_SECRET_ACCESS_KEY
environ["MLFLOW_S3_ENDPOINT_URL"] = MLFLOW_S3_ENDPOINT_URL
environ["MLFLOW_ENABLE_ARTIFACTS_PROGRESS_BAR"] = "false"

In [3]:
import mlflow

if MLFLOW_URL is not None:
    environ["MLFLOW_S3_ENDPOINT_URL"] = MLFLOW_S3_ENDPOINT_URL
    mlflow.set_tracking_uri(MLFLOW_URL)

In [4]:
import mlflow
import mlflow.tensorflow
import tensorflow as tf

if MLFLOW_RUN_ID is None:
    client = mlflow.tracking.MlflowClient()

    experiments = client.search_experiments(
        filter_string = "name = 'ML Quantum Entanglement'"
    )
    experiment_id = experiments[0].experiment_id

    filter_string = (
    f"tags.airflow_dag_run_id = '{AIRFLOW_DAG_RUN_ID}' and "
    f"tags.pipeline_step = 'training'"
    )   
    runs = client.search_runs(
        experiment_ids=[experiment_id], 
        filter_string=filter_string
    )
    run_id = runs[0].info.run_id
else :
    run_id = MLFLOW_RUN_ID
model_uri = f"runs:/{run_id}/model"
model = mlflow.pyfunc.load_model(model_uri)

2024-07-17 17:30:40.516001: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-17 17:30:40.546838: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-17 17:30:41.035343: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-07-17 17:30:41.763145: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:282] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2024-07-17 17:30:41.763162: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:134] retrieving 

In [11]:
import numpy as np

from simulation_utils import create_random_bell_phi, \
    create_random_bell_psi, create_random_separable, \
        create_entangled_werner_state_with_chsh_violation, \
            create_entangled_werner_state_without_chsh_violation, \
                create_not_entangled_werner_state_dm, \
                    create_mixed_not_entangled_state, flatten_density_matrix

sim_functions = [
    create_random_bell_phi, 
    create_random_bell_psi,
    create_random_separable, 
    create_entangled_werner_state_with_chsh_violation, 
    create_entangled_werner_state_without_chsh_violation, 
    create_not_entangled_werner_state_dm,
    create_mixed_not_entangled_state
]

for sim_func in sim_functions:

    flatten_dm = flatten_density_matrix(sim_func())
    state_input = np.array(flatten_dm)
    state_input = np.expand_dims(flatten_dm, axis=0)

    prediction = model.predict(state_input)
    print(f"Witness model: {WITNESS_NAME} Simulation function: {sim_func.__name__} Entanglement prediction: {prediction}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
Witness model: CHSH Simulation function: create_random_bell_phi Entanglement prediction: [[0.9999979]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Witness model: CHSH Simulation function: create_random_bell_psi Entanglement prediction: [[0.00846853]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
Witness model: CHSH Simulation function: create_random_separable Entanglement prediction: [[2.443367e-08]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
Witness model: CHSH Simulation function: create_entangled_werner_state_with_chsh_violation Entanglement prediction: [[0.9999998]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
Witness model: CHSH Simulation function: create_entangled_werner_state_without_chsh_violation Entanglement prediction: [[0.9998808]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
Witness model: CHSH Simulation function: create_not_entangled_werner_state_dm Entanglement prediction: [[1.2910277e-05]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
Witness model: CHSH Simulation functi